<a href="https://colab.research.google.com/github/ryrahman-arch/NGG_6050/blob/main/Rahman_LATERmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd

# Using pandas to make a data frame for simulated data based off the conditions
# provided in the Matlab Box data folder
# Set random seed for reproducibility
np.random.seed(42)

# Parameters
num_samples = 70
subject_ids = [f"{chr(i)}{chr(j)}" for i in range(65, 91) for j in range(65, 91)]
np.random.shuffle(subject_ids)

# Assign groups
groups = np.random.choice(['both', 'fast (s)', 'accurate (a)'], num_samples)

# Reaction time conditions
conditions = ['strong with feedback', 'strong with no feedback', 'weak with feedback', 'weak with no feedback']
reaction_times = []

for _ in range(num_samples):
    condition = np.random.choice(conditions)
    if condition.startswith('strong'):
        if 'feedback' in condition:
            reaction_time = np.random.normal(loc=300, scale=50)  # Strong with feedback
        else:
            reaction_time = np.random.normal(loc=320, scale=50)  # Strong with no feedback
    else:
        if 'feedback' in condition:
            reaction_time = np.random.normal(loc=400, scale=50)  # Weak with feedback
        else:
            reaction_time = np.random.normal(loc=420, scale=50)  # Weak with no feedback

    reaction_times.append(reaction_time)

# Create a DataFrame
data = pd.DataFrame({
    'SubjectID': subject_ids[:num_samples],
    'Group': groups,
    'Condition': [np.random.choice(conditions) for _ in range(num_samples)],
    'ReactionTime': reaction_times
})

# Remove outliers based on RT bins (0 to 1.2 seconds)
rt_bins = np.arange(0, 1.21, 0.02)
filtered_data = data[(data['ReactionTime'] >= 0) & (data['ReactionTime'] <= 1200)]

# Remove outliers based on 1/RT bins (0 to 10.0)
filtered_data['InverseRT'] = 1 / (filtered_data['ReactionTime'] / 1000)  # Convert ms to seconds
rrt_bins = np.arange(0, 10.1, 0.2)
filtered_data = filtered_data[(filtered_data['InverseRT'] >= 0) & (filtered_data['InverseRT'] <= 10)]

# Drop the 'InverseRT' column as it's no longer needed
filtered_data = filtered_data.drop(columns=['InverseRT'])

# Display the first few rows of the filtered DataFrame
print(filtered_data.head())

# Optionally, save to a CSV file
# filtered_data.to_csv('filtered_reaction_time_data.csv', index=False)



  SubjectID         Group                Condition  ReactionTime
0        YR          both  strong with no feedback    126.091539
1        LQ          both     strong with feedback    386.586847
2        OF          both  strong with no feedback    446.853626
3        SZ      fast (s)  strong with no feedback    371.533693
4        WH  accurate (a)       weak with feedback    271.837069


In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Set random seed for reproducibility
np.random.seed(42)

# Parameters
num_samples = 70
subject_ids = [f"{chr(i)}{chr(j)}" for i in range(65, 91) for j in range(65, 91)]
np.random.shuffle(subject_ids)

# Assign groups
groups = np.random.choice(['both', 'fast (s)', 'accurate (a)'], num_samples)

# Reaction time conditions
conditions = ['strong with feedback', 'strong with no feedback', 'weak with feedback', 'weak with no feedback']
reaction_times = []

for _ in range(num_samples):
    condition = np.random.choice(conditions)
    if condition.startswith('strong'):
        if 'feedback' in condition:
            reaction_time = np.random.normal(loc=300, scale=50)  # Strong with feedback
        else:
            reaction_time = np.random.normal(loc=320, scale=50)  # Strong with no feedback
    else:
        if 'feedback' in condition:
            reaction_time = np.random.normal(loc=400, scale=50)  # Weak with feedback
        else:
            reaction_time = np.random.normal(loc=420, scale=50)  # Weak with no feedback

    reaction_times.append(reaction_time)

# Create a DataFrame
data = pd.DataFrame({
    'SubjectID': subject_ids[:num_samples],
    'Group': groups,
    'Condition': [np.random.choice(conditions) for _ in range(num_samples)],
    'ReactionTime': reaction_times
})

# Remove outliers based on RT bins (0 to 1.2 seconds)
rt_bins = np.arange(0, 1.21, 0.02)
filtered_data = data[(data['ReactionTime'] >= 0) & (data['ReactionTime'] <= 1200)]

# Remove outliers based on 1/RT bins (0 to 10.0)
filtered_data['InverseRT'] = 1 / (filtered_data['ReactionTime'] / 1000)  # Convert ms to seconds
rrt_bins = np.arange(0, 10.1, 0.2)
filtered_data = filtered_data[(filtered_data['InverseRT'] >= 0) & (filtered_data['InverseRT'] <= 10)]

# Drop the 'InverseRT' column as it's no longer needed
filtered_data = filtered_data.drop(columns=['InverseRT'])

# Define the objective function (negative log-likelihood)
def laterErrFcn(data):
    # Calculate log-likelihoods assuming a normal distribution
    ll = 0
    for _, row in data.iterrows():
        if row['Condition'].startswith('strong'):
            mu, sigma = (300, 50) if 'feedback' in row['Condition'] else (320, 50)
        else:
            mu, sigma = (400, 50) if 'feedback' in row['Condition'] else (420, 50)

        # Compute log-likelihood for the reaction time
        ll += norm.logpdf(row['ReactionTime'], loc=mu, scale=sigma)

    return -ll  # Return the negative log-likelihood

# Calculate the error
error = laterErrFcn(filtered_data)

# Display the first few rows of the filtered DataFrame and the error
print(filtered_data.head())
print(f"Negative Log-Likelihood Error: {error}")

  SubjectID         Group                Condition  ReactionTime
0        YR          both  strong with no feedback    126.091539
1        LQ          both     strong with feedback    386.586847
2        OF          both  strong with no feedback    446.853626
3        SZ      fast (s)  strong with no feedback    371.533693
4        WH  accurate (a)       weak with feedback    271.837069
Negative Log-Likelihood Error: 446.9669104996816


In [6]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize

# trying again but fmincon?
# Set random seed for reproducibility
np.random.seed(42)

# Parameters
num_samples = 70
subject_ids = [f"{chr(i)}{chr(j)}" for i in range(65, 91) for j in range(65, 91)]
np.random.shuffle(subject_ids)

# Assign groups
groups = np.random.choice(['both', 'fast (s)', 'accurate (a)'], num_samples)

# Reaction time conditions
conditions = ['strong with feedback', 'strong with no feedback', 'weak with feedback', 'weak with no feedback']
reaction_times = []

for _ in range(num_samples):
    condition = np.random.choice(conditions)
    if condition.startswith('strong'):
        if 'feedback' in condition:
            reaction_time = np.random.normal(loc=300, scale=50)
        else:
            reaction_time = np.random.normal(loc=320, scale=50)
    else:
        if 'feedback' in condition:
            reaction_time = np.random.normal(loc=400, scale=50)
        else:
            reaction_time = np.random.normal(loc=420, scale=50)

    reaction_times.append(reaction_time)

# Create a DataFrame
data = pd.DataFrame({
    'SubjectID': subject_ids[:num_samples],
    'Group': groups,
    'Condition': [np.random.choice(conditions) for _ in range(num_samples)],
    'ReactionTime': reaction_times
})

# Remove outliers based on RT bins (0 to 1.2 seconds)
filtered_data = data[(data['ReactionTime'] >= 0) & (data['ReactionTime'] <= 1200)]

# Remove outliers based on 1/RT bins (0 to 10.0)
filtered_data['InverseRT'] = 1 / (filtered_data['ReactionTime'] / 1000)  # Convert ms to seconds
filtered_data = filtered_data[(filtered_data['InverseRT'] >= 0) & (filtered_data['InverseRT'] <= 10)]
filtered_data = filtered_data.drop(columns=['InverseRT'])

# Define the objective function (negative log-likelihood)
def laterErrFcn(params, data):
    mu, sigma = params
    ll = 0
    for _, row in data.iterrows():
        if row['Condition'].startswith('strong'):
            condition_mu = mu if 'feedback' in row['Condition'] else mu + 20
            condition_sigma = sigma
        else:
            condition_mu = mu + 100 if 'feedback' in row['Condition'] else mu + 120
            condition_sigma = sigma

        # Compute log-likelihood for the reaction time
        ll += norm.logpdf(row['ReactionTime'], loc=condition_mu, scale=condition_sigma)

    return -ll  # Return the negative log-likelihood

# Initial guesses for mu and sigma
initial_params = [300, 50]  # Initial values for mu and sigma

# Set bounds: (0.001, 1000) for both mu and sigma
bounds = [(0.001, 1000), (0.001, 1000)]

# Optimize parameters
result = minimize(laterErrFcn, initial_params, args=(filtered_data,), bounds=bounds)

# Extract optimized parameters
optimized_mu, optimized_sigma = result.x
error = result.fun  # The optimized error (negative log-likelihood)

# Display the first few rows of the filtered DataFrame and the optimization results
print(filtered_data.head())
print(f"Optimized Mu: {optimized_mu}")
print(f"Optimized Sigma: {optimized_sigma}")
print(f"Negative Log-Likelihood Error: {error}")


  SubjectID         Group                Condition  ReactionTime
0        YR          both  strong with no feedback    126.091539
1        LQ          both     strong with feedback    386.586847
2        OF          both  strong with no feedback    446.853626
3        SZ      fast (s)  strong with no feedback    371.533693
4        WH  accurate (a)       weak with feedback    271.837069
Optimized Mu: 301.7219600910706
Optimized Sigma: 88.14058037746659
Negative Log-Likelihood Error: 412.84954638225565


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize, differential_evolution

# Set random seed for reproducibility
np.random.seed(42)

# Parameters
num_samples = 70
subject_ids = [f"{chr(i)}{chr(j)}" for i in range(65, 91) for j in range(65, 91)]
np.random.shuffle(subject_ids)

# Assign groups
groups = np.random.choice(['both', 'fast (s)', 'accurate (a)'], num_samples)

# Reaction time conditions
conditions = ['strong with feedback', 'strong with no feedback', 'weak with feedback', 'weak with no feedback']
reaction_times = []

for _ in range(num_samples):
    condition = np.random.choice(conditions)
    if condition.startswith('strong'):
        if 'feedback' in condition:
            reaction_time = np.random.normal(loc=300, scale=50)
        else:
            reaction_time = np.random.normal(loc=320, scale=50)
    else:
        if 'feedback' in condition:
            reaction_time = np.random.normal(loc=400, scale=50)
        else:
            reaction_time = np.random.normal(loc=420, scale=50)

    reaction_times.append(reaction_time)

# Create a DataFrame
data = pd.DataFrame({
    'SubjectID': subject_ids[:num_samples],
    'Group': groups,
    'Condition': [np.random.choice(conditions) for _ in range(num_samples)],
    'ReactionTime': reaction_times
})

# Remove outliers based on RT bins (0 to 1.2 seconds)
filtered_data = data[(data['ReactionTime'] >= 0) & (data['ReactionTime'] <= 1200)]

# Remove outliers based on 1/RT bins (0 to 10.0)
filtered_data['InverseRT'] = 1 / (filtered_data['ReactionTime'] / 1000)  # Convert ms to seconds
filtered_data = filtered_data[(filtered_data['InverseRT'] >= 0) & (filtered_data['InverseRT'] <= 10)]
filtered_data = filtered_data.drop(columns=['InverseRT'])

# Define the objective function (negative log-likelihood)
def laterErrFcn(params, data):
    mu, sigma = params
    ll = 0
    for _, row in data.iterrows():
        if row['Condition'].startswith('strong'):
            condition_mu = mu if 'feedback' in row['Condition'] else mu + 20
            condition_sigma = sigma
        else:
            condition_mu = mu + 100 if 'feedback' in row['Condition'] else mu + 120
            condition_sigma = sigma

        # Compute log-likelihood for the reaction time
        ll += norm.logpdf(row['ReactionTime'], loc=condition_mu, scale=condition_sigma)

    return -ll  # Return the negative log-likelihood

# Define bounds for optimization
bounds = [(0.001, 1000), (0.001, 1000)]  # (mu, sigma) bounds

# Run the global search optimization using differential evolution
result = differential_evolution(laterErrFcn, bounds, args=(filtered_data,), maxiter=3000, popsize=15)

# Extract optimized parameters
optimized_mu, optimized_sigma = result.x
error = result.fun  # The optimized error (negative log-likelihood)

# Display the first few rows of the filtered DataFrame and the optimization results
print(filtered_data.head())
print(f"Optimized Mu: {optimized_mu}")
print(f"Optimized Sigma: {optimized_sigma}")
print(f"Negative Log-Likelihood Error: {error}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, kstest

# Assuming the previous code has been run and optimized parameters are available
# optimized_mu, optimized_sigma, and filtered_data from previous steps

# Step 1: Visualize the fitted distributions
def plot_fitted_distribution(data, mu, sigma):
    plt.figure(figsize=(10, 6))

    # Histogram of actual reaction times
    plt.hist(data['ReactionTime'], bins=30, density=True, alpha=0.6, color='gray', label='Observed Data')

    # Generate x values for the fitted curve
    x = np.linspace(0, 1200, 1000)

    # Plot fitted normal distribution
    for condition in ['strong with feedback', 'strong with no feedback', 'weak with feedback', 'weak with no feedback']:
        if 'feedback' in condition:
            fitted_mu = mu
        else:
            fitted_mu = mu + 20  # Adjust as per the previous logic
        fitted_sigma = sigma

        # Plot the fitted distribution for this condition
        plt.plot(x, norm.pdf(x, loc=fitted_mu, scale=fitted_sigma), label=f'Fitted {condition}')

    plt.title('Fitted Distributions vs Observed Data')
    plt.xlabel('Reaction Time (ms)')
    plt.ylabel('Density')
    plt.legend()
    plt.grid()
    plt.show()

# Step 2: Residual analysis
def residual_analysis(data, mu, sigma):
    residuals = []
    for _, row in data.iterrows():
        if row['Condition'].startswith('strong'):
            condition_mu = mu if 'feedback' in row['Condition'] else mu + 20
            condition_sigma = sigma
        else:
            condition_mu = mu + 100 if 'feedback' in row['Condition'] else mu + 120
            condition_sigma = sigma

        predicted = norm.pdf(row['ReactionTime'], loc=condition_mu, scale=condition_sigma)
        residuals.append(row['ReactionTime'] - predicted)

    plt.figure(figsize=(10, 6))
    plt.scatter(data['ReactionTime'], residuals)
    plt.axhline(0, color='red', linestyle='--')
    plt.title('Residuals Analysis')
    plt.xlabel('Observed Reaction Time (ms)')
    plt.ylabel('Residuals')
    plt.grid()
    plt.show()

# Step 3: Goodness-of-Fit Test (Kolmogorov-Smirnov Test)
def goodness_of_fit_test(data, mu, sigma):
    ks_statistic, p_value = kstest(data['ReactionTime'], 'norm', args=(mu, sigma))
    print(f"KS Statistic: {ks_statistic}, P-value: {p_value}")
    if p_value > 0.05:
        print("Fail to reject the null hypothesis: The fit is reasonable.")
    else:
        print("Reject the null hypothesis: The fit is not reasonable.")

# Run evaluations
plot_fitted_distribution(filtered_data, optimized_mu, optimized_sigma)
residual_analysis(filtered_data, optimized_mu, optimized_sigma)
goodness_of_fit_test(filtered_data, optimized_mu, optimized_sigma)
